In [ ]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
from motion_generation.MoE_Z import MoE as MoE_Z
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.GRU_Z import GRU as GRU_Z
from motion_generation.LSTM import LSTM
from motion_generation.LSTM_Z import LSTM as LSTM_Z

from motion_generation.MotionGeneration import MotionGenerationModel as MoGen
from motion_generation.MotionGenerationEmbedd import MotionGenerationModel as MoGenZ
from motion_generation.MotionGenerationVAE import MotionGenerationModel as MoGenVAE
from motion_generation.MotionGenerationVAE_Embedd import MotionGenerationModel as MoGenVAE_Z

from MLP import MLP
from MLP_Adversarial import MLP_ADV
from MLP_MIX import MLP_MIX
from RBF import RBF
from VAE import VAE

from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2
from cytoolz import concat, sliding_window, accumulate
from operator import add
from collections import OrderedDict
import torch
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib
import random

In [ ]:
config = {
    "hidden_dim": 256,
    "k": 256,
    "z_dim": 256,
    "lr": 1e-4,
    "batch_size": 16,
    "keep_prob": 0,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.AdamW,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":64,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":.3,
    "autoregress_ep":20,
    "autoregress_max_prob":1,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [ ]:
def getFilesNames(file_paths, data_path, MAX_FILES=-1):
    for dname, dirs, files in os.walk(data_path):
        for i, file in enumerate(files):
            file_paths.append(os.path.join(dname, file))
            if MAX_FILES > 0 and i >= MAX_FILES:
                break
    return file_paths

data_path = "/home/nuoc/Documents/MEX/data/data/Dataset_R1_Two_1"
data_path2 = "/home/nuoc/Documents/MEX/data/data/Dataset_R2_Two_1"
# data_path3 = "/home/nuoc/Documents/MEX/data/data/Dataset_R3_Two_1"
# data_path4 = "/home/nuoc/Documents/MEX/data/data/Dataset_R4_Two_1"
file_paths = getFilesNames([],data_path)
file_paths2 = getFilesNames([],data_path2)
# file_paths3 = getFilesNames([],data_path3)
# file_paths4 = getFilesNames([],data_path4)

print(len(file_paths))
print(len(file_paths2))





In [ ]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
pose_label_feature = ["chainPos", "isLeft", "geoDistanceNormalised"]
target_features = ["targetPosition", "targetRotation"]
features = phase_features + pose_features + cost_features + target_features
clips = []
feature_dims = {}


In [ ]:
data = F.process_data_multithread(file_paths, features)
data2 = F.process_data_multithread(file_paths2, features)

In [ ]:
pose_labels1 = F.process_data_multithread([file_paths[0]], pose_label_feature)
pose_labels2 = F.process_data_multithread([file_paths2[0]], pose_label_feature)

In [ ]:
obj = {"data":data, "data2":data2, "pose_label1":pose_labels1, "pose_label2":pose_labels2}
F.save(obj, filename="transfer_learning_set_R1-R2_Two_wTarget", path="/home/nuoc/Documents/MEX/data/")

In [ ]:
obj = F.load("/home/nuoc/Documents/MEX/data/transfer_learning_set_R1-R2_Two_wTarget.pbz2")
data = obj["data"]
data2 = obj["data2"]

In [ ]:
pose_labels1 = obj["pose_label1"]
pose_labels2 = obj["pose_label2"]

In [ ]:
feature_dims = data[0][1]
feature_dims2 = data2[0][1]
clips = [np.copy(d[0]) for d in data]
clips2 = [np.copy(d[0]) for d in data2]

pose_labels1 = pose_labels1[0]
pose_labels2 = pose_labels2[0]

In [ ]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
pose_dim2 = sum([feature_dims2[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
target_dim = sum([feature_dims[feature] for feature in target_features])
print(phase_dim, " ", cost_dim, " ", target_dim)

In [ ]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])

x_tensors2 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips2])
y_tensors2 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips2])


pose_data1 = x_tensors[:,  :,  phase_dim:phase_dim+pose_dim]
pose_data2 = x_tensors2[:, :, phase_dim:phase_dim+pose_dim2]
pose_data = torch.cat((pose_data1, pose_data2), dim=2)

In [ ]:
dataset_p = TensorDataset(pose_data, pose_data)
dataset_p1 = TensorDataset(pose_data1, pose_data1)
dataset_p2 = TensorDataset(pose_data2, pose_data2)
datasetR1 = TensorDataset(x_tensors, y_tensors)
datasetR2 = TensorDataset(x_tensors2, y_tensors2)

N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio

train_set_p, val_set_p, test_set_p = random_split(dataset_p, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p1, val_set_p1, test_set_p1 = random_split(dataset_p1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2, val_set_p2, test_set_p2 = random_split(dataset_p2, [val_ratio*2, val_ratio, train_ratio-test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2_F, val_set_p2_F, test_set_p2_F = random_split(dataset_p2, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR1, val_setR1, test_setR1 = random_split(datasetR1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2, val_setR2, test_setR2 = random_split(datasetR2, [val_ratio, val_ratio, train_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2_F, val_setR2_F, test_setR2_F = random_split(datasetR2, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))

In [ ]:
def extract_targets(train_set, val_set, test_set, target_dim):
    t1, t2, t3, t33 = [], [], [], []
    for i in range(len(train_set)):
        x = train_set[i][0]
        y = train_set[i][1]
        t1.append((x[:, :-target_dim], y[:, :-target_dim]))
    for i in range(len(val_set)):
        x = val_set[i][0]
        y = val_set[i][1]
        t2.append((x[:, :-target_dim:], y[:, :-target_dim:]))
    for i in range(len(test_set)):
        x = test_set[i][0]
        y = test_set[i][1]
        t3.append((x[:, :-target_dim:], y[:, :-target_dim:]))
        t33.append((x[:, :-target_dim:], y))
    return t1, t2, t3, t33




In [ ]:
t1, v1, te1, te11 = extract_targets(train_set=train_setR1, val_set=val_setR1, test_set=test_setR1, target_dim=target_dim)
t2, v2, te2, te22 = extract_targets(train_set=train_setR2, val_set=val_setR2, test_set=test_setR2, target_dim=target_dim)

# AE ONLY

In [ ]:
h_dim = train_set_p1[0][0].shape[0]
w_dim = train_set_p1[0][0].shape[1]

ae_name1 = "AE_R1"
# ae1 = MLP_ADV(config=config, dimensions=[pose_dim], h_dim=h_dim, w_dim=w_dim,
#                  pos_dim=feature_dims["pos"], rot_dim=feature_dims["rotMat2"], vel_dim=feature_dims["velocity"],
#                  train_set=train_set_p1, val_set=val_set_p1, test_set=test_set_p1, name=ae_name1)
#
# ae1 = MLP_ADV.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.3/AE_R1/0.001.256.pbz2")
# ae1.name = ae_name1
def load_ae(path):
    with bz2.BZ2File(path, "rb") as f:
        obj = pickle.load(f)
    model = MLP_ADV(config=obj["config"], single_module=obj["single_module"], pose_labels=obj["pose_labels"],
                h_dim=h_dim, w_dim=w_dim,
                name=obj["name"], dimensions=obj["dimensions"])

    model.encoder.load_state_dict(obj["encoder"])
    model.decoder.load_state_dict(obj["decoder"])
    model.convDiscriminator.load_state_dict(obj["discriminator"])
    model.pos_dim = obj["dims"][0]
    model.rot_dim = obj["dims"][1]
    model.vel_dim = obj["dims"][2]
    return model

ae1 = load_ae("/home/nuoc/Documents/MEX/models/version_0.3/AE_R1/0.001.256.pbz2")

In [ ]:
ae_name2 = "AE_R2"
# ae2 = MLP_ADV(config=config, dimensions=[pose_dim2],h_dim=h_dim, w_dim=w_dim,
#                  pos_dim=feature_dims2["pos"], rot_dim=feature_dims2["rotMat2"], vel_dim=feature_dims2["velocity"],
#                  train_set=train_set_p2_F, val_set=val_set_p2_F, test_set=test_set_p2_F, name=ae_name2)
#
# ae2 = MLP_ADV.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.3/AE_R2/0.002.256.pbz2")
ae2 = load_ae("/home/nuoc/Documents/MEX/models/version_0.3/AE_R2/0.002.256.pbz2")
# ae2.name = ae_name2

In [ ]:
ae1.test_set = test_set_p1
ae2.test_set = test_set_p2


In [ ]:
trainer = pl.Trainer()
trainer.test(ae1)
trainer.test(ae2)

In [ ]:
def fit(model, name, version="0.1", MIN_EPOCHS=20, MAX_EPOCHS=100, useEarlyStopping=False, patience=10):
    if useEarlyStopping:
        earlystopping = EarlyStopping(monitor="avg_val_loss",patience=patience)
        callbacks = [earlystopping]
    else:
        callbacks = []
    logger=TensorBoardLogger(save_dir="logs/", name=name, version=version)

    trainer = pl.Trainer(logger=logger, gpus=1, precision=16)
    trainer.test(model)

    trainer = pl.Trainer(
        default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
        gpus=1, precision=16,
        callbacks= callbacks,
        min_epochs=MIN_EPOCHS,
        logger=logger,
        max_epochs=MAX_EPOCHS,
        stochastic_weight_avg=True
    )
    trainer.fit(model)
    trainer.test(model)

In [ ]:
fit(ae1, ae_name1, MAX_EPOCHS=300, useEarlyStopping=True)

In [ ]:
fit(ae2, ae_name2, version="0.2", MAX_EPOCHS=300, useEarlyStopping=True)

In [ ]:
clean_checkpoints(os.path.join(MODEL_PATH,ae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,ae_name2))

ae1.save_checkpoint(best_val_loss=0.001)
ae2.save_checkpoint(best_val_loss=0.002)

In [ ]:
generate_animation_ae(model=ae1, test_set=test_set_p1, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+ae_name1)
generate_animation_ae(model=ae2, test_set=test_set_p2_F, feature_dims=feature_dims2,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+ae_name2)


In [ ]:
model_name = "MLP_MoE_R1_ADV"
model_name2 = "MLP_MoE_R2_ADV"

featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "target_dim":target_dim,
    "g_input_dim": config["z_dim"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "target_dim": target_dim,
    "g_input_dim": config["z_dim"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }

in_slice = [phase_dim, pose_dim, cost_dim, target_dim]
in_slice2 = [phase_dim, pose_dim2, cost_dim,target_dim]

out_slice = [phase_dim, config["k"], cost_dim]

temp = MLP_ADV(config=config, dimensions=[pose_dim], h_dim=h_dim, w_dim=w_dim,)
temp2 = MLP_ADV(config=config, dimensions=[pose_dim2],h_dim=h_dim, w_dim=w_dim,)

pose_encoder = temp
pose_encoder2 = temp2
pose_encoder.encoder.load_state_dict(ae1.encoder.state_dict())
pose_encoder.decoder.load_state_dict(ae1.decoder.state_dict())
pose_encoder.convDiscriminator.load_state_dict(ae1.convDiscriminator.state_dict())

pose_encoder2.encoder.load_state_dict(ae2.encoder.state_dict())
pose_encoder2.decoder.load_state_dict(ae2.decoder.state_dict())
pose_encoder2.convDiscriminator.load_state_dict(ae2.convDiscriminator.state_dict())

In [ ]:
middle_layer = torch.nn.Sequential()
model1 = MoGen(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim, use_advLoss=True,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_setR1, val_set=val_setR1, test_set=val_setR1+test_setR1,
                                 name=model_name
                                   )

In [ ]:
model2 = MoGen(config=config, Model=MoE, pose_autoencoder=pose_encoder2, middle_layer=middle_layer,
                                 feature_dims=featureDim2, use_advLoss=True,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=train_setR2, val_set=val_setR2, test_set=val_setR2+test_setR2,
                                 name=model_name2
                                   )

In [ ]:
model2_F = MoGen(config=config, Model=MoE, pose_autoencoder=pose_encoder2, middle_layer=middle_layer,
                                 feature_dims=featureDim2, use_advLoss=True,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=train_setR2_F, val_set=val_setR2_F, test_set=val_setR2_F+test_setR2_F,
                                 name=model_name2
                                   )

In [ ]:
fit(model1, version="0.4", MAX_EPOCHS=150, name=model_name)


In [ ]:
model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

fit(model2, name=model_name2, version="0.2", MAX_EPOCHS=100)


In [ ]:
model2_F.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2_F.generationModel.load_state_dict(model1.generationModel.state_dict())

fit(model2_F, name=model_name2, version="full", MAX_EPOCHS=100)


In [ ]:
clean_checkpoints(os.path.join(MODEL_PATH,model_name))
clean_checkpoints(os.path.join(MODEL_PATH,model_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)
model2_F.save_checkpoint(best_val_loss=0.001)

In [ ]:
generate_animation(model=model1, test_set=test_setR1, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name)
generate_animation(model=model2, test_set=test_setR2, feature_dims=feature_dims2,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name2)
generate_animation(model=model2_F, test_set=test_setR2, feature_dims=feature_dims2,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name2+"F")


# MLP MIX - Z as embedding

In [ ]:
config["z_dim"]=128
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "target_dim":target_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "target_dim": target_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }


in_slice = [phase_dim, pose_dim, cost_dim, target_dim]
in_slice2 = [phase_dim, pose_dim2, cost_dim,target_dim]

out_slice = [phase_dim, config["k"], cost_dim]

mlpmix_name1 = "MLPMIX-R1-Z-Concat"
mlpmix1 = MLP_MIX(config=config, input_dims=[pose_dim])
temp = MLP_ADV(config=config, dimensions=[pose_dim], h_dim=h_dim, w_dim=w_dim,)

mlpmix1.active_models[0] = temp
mlpmix1.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
mlpmix1.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = mlpmix1.active_models[0]
middle_layer = mlpmix1.cluster_model
pose_encoder.convDiscriminator.load_state_dict(ae1.convDiscriminator.state_dict())

In [ ]:
model1 = MoGenZ(config=config, Model=MoE_Z, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim, use_advLoss=True,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_setR1, val_set=val_setR1, test_set=test_setR1+val_setR1,
                                 name=mlpmix_name1
                                   )


In [ ]:
fit(model1, mlpmix_name1, version="0.3",MAX_EPOCHS=150)

In [ ]:
mlpmix_name2 = "MLPMIX-R2-Z-Concat-Reduced"
mlpmix2 = MLP_MIX(config=config, input_dims=[pose_dim2])
temp = MLP_ADV(config=config, dimensions=[pose_dim2], h_dim=h_dim, w_dim=w_dim,)

in_slice2 = [phase_dim, pose_dim2, cost_dim,target_dim]
out_slice = [phase_dim, config["k"], cost_dim]

temp.encoder.load_state_dict(ae2.encoder.state_dict())
temp.decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = temp
middle_layer = mlpmix2.cluster_model
pose_encoder.convDiscriminator.load_state_dict(ae2.convDiscriminator.state_dict())

middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MoGenZ(config=config, Model=MoE_Z, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,use_advLoss=True,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=train_setR2, val_set=val_setR2, test_set=val_setR2+test_setR2,
                                 name=mlpmix_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

In [ ]:
fit(model2, mlpmix_name2, version="0.2", MAX_EPOCHS=100)

In [ ]:
clean_checkpoints(os.path.join(MODEL_PATH,mlpmix_name1))
clean_checkpoints(os.path.join(MODEL_PATH,mlpmix_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [ ]:
generate_animation(model=model1, test_set=test_setR1, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/version0.3/"+mlpmix_name1)
generate_animation(model=model2, test_set=test_setR2, feature_dims=feature_dims2,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/version0.3/"+mlpmix_name2)

# RBF -  Z as embedding

In [ ]:
config["z_dim"] = 256
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "target_dim":target_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "target_dim": target_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }

rbf_name1 = "RBF-R1-Z-Concat"
rbf1 = RBF(config=config, input_dims=[pose_dim],
           pos_dim=[featureDim["pos_dim"]], rot_dim=[featureDim["rot_dim"]], vel_dim=[featureDim["vel_dim"]],
           train_set=train_set_p1, val_set=val_set_p1, test_set=test_set_p1,
           name="RBF-z-concat")

temp = MLP_ADV(config=config, dimensions=[pose_dim], h_dim=h_dim, w_dim=w_dim,)

rbf1.active_models[0] = temp
rbf1.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
rbf1.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())
rbf1.active_models[0].convDiscriminator.load_state_dict(ae1.convDiscriminator.state_dict())

In [ ]:
fit(rbf1, rbf_name1, version="0.1", MAX_EPOCHS=100)

In [ ]:
in_slice = [phase_dim, pose_dim, cost_dim,target_dim]
out_slice = [phase_dim, config["k"], cost_dim]

pose_encoder = rbf1.active_models[0]
middle_layer = rbf1.cluster_model
middle_layer.requires_grad_(False)

model1 = MoGenZ(config=config, Model=MoE_Z, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim, use_advLoss=True,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_setR1, val_set=val_setR1, test_set=test_setR1+val_setR1,
                                 name=rbf_name1
                                   )

In [ ]:
fit(model1, rbf_name1, version="0.2",MAX_EPOCHS=150)

In [ ]:
rbf_name2 = "RBF-R2-Z-Concat-Reduced"
rbf2 = RBF(config=config, input_dims=[pose_dim2])
temp = MLP_ADV(config=config, dimensions=[pose_dim2], h_dim=h_dim, w_dim=w_dim,)

in_slice2 = [phase_dim, pose_dim2, cost_dim,target_dim]
out_slice = [phase_dim, config["k"], cost_dim]

rbf2.active_models[0] = temp
rbf2.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
rbf2.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = rbf2.active_models[0]
middle_layer = rbf2.cluster_model
pose_encoder.convDiscriminator.load_state_dict(ae2.convDiscriminator.state_dict())

middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MoGenZ(config=config, Model=MoE_Z, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2, use_advLoss=True,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=train_setR2, val_set=val_setR2, test_set=val_setR2+test_setR2,
                                 name=rbf_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

In [ ]:
fit(model2, rbf_name2, version="0.2", MAX_EPOCHS=100)

In [ ]:
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name1))
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [ ]:
# generate_animation(model=model1, test_set=test_setR1, feature_dims=feature_dims,
#                    template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
#                    output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/version0.3/"+rbf_name1)
generate_animation(model=model2, test_set=test_setR2, feature_dims=feature_dims2,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/version0.3/"+rbf_name2+"After")





# vae -  Z as embedding

In [ ]:
config["z_dim"] = 256
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "target_dim":target_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "target_dim": target_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }

vae_name1 = "VAE-R1-Z-Concat"
vae1 = VAE(config=config, input_dims=[pose_dim],
           pos_dim=[featureDim["pos_dim"]], rot_dim=[featureDim["rot_dim"]], vel_dim=[featureDim["vel_dim"]],
           train_set=train_set_p1, val_set=val_set_p1, test_set=test_set_p1,
           name="VAE-z-concat")

temp = MLP_ADV(config=config, dimensions=[pose_dim], h_dim=h_dim, w_dim=w_dim,)

vae1.active_models[0] = temp
vae1.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
vae1.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())
vae1.active_models[0].convDiscriminator.load_state_dict(ae1.convDiscriminator.state_dict())

In [ ]:
fit(vae1, vae_name1, version="0.1", MAX_EPOCHS=100)

In [ ]:
in_slice = [phase_dim, pose_dim, cost_dim,target_dim]
out_slice = [phase_dim, config["k"], cost_dim]

pose_encoder = vae1.active_models[0]
middle_layer = vae1.cluster_model
middle_layer.requires_grad_(False)

model1 = MoGenVAE_Z(config=config, Model=MoE_Z, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim, use_advLoss=True,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_setR1, val_set=val_setR1, test_set=test_setR1+val_setR1,
                                 name=vae_name1
                                   )

In [ ]:
fit(model1, vae_name1, version="0.1",MAX_EPOCHS=150)

In [ ]:
vae_name2 = "VAE-R2-Z-Concat-Reduced"
vae2 = VAE(config=config, input_dims=[pose_dim2])
temp = MLP_ADV(config=config, dimensions=[pose_dim2], h_dim=h_dim, w_dim=w_dim,)

in_slice2 = [phase_dim, pose_dim2, cost_dim,target_dim]
out_slice = [phase_dim, config["k"], cost_dim]

vae2.active_models[0] = temp
vae2.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
vae2.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = vae2.active_models[0]
middle_layer = vae2.cluster_model
pose_encoder.convDiscriminator.load_state_dict(ae2.convDiscriminator.state_dict())

middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MoGenVAE_Z(config=config, Model=MoE_Z, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2, use_advLoss=True,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=train_setR2, val_set=val_setR2, test_set=val_setR2+test_setR2,
                                 name=vae_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

In [ ]:
fit(model2, vae_name2, version="0.3", MAX_EPOCHS=100)

In [ ]:
clean_checkpoints(os.path.join(MODEL_PATH,vae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,vae_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [ ]:
generate_animation(model=model1, test_set=test_setR1, feature_dims=feature_dims, use_vae=True,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/version0.3/"+vae_name1)
generate_animation(model=model2, test_set=test_setR2, feature_dims=feature_dims2,use_vae=True,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/version0.3/"+vae_name2+"After")




In [ ]:
def setVec3(struct, vec):
    struct["x"] = vec[0].item()
    struct["y"] = vec[1].item()
    struct["z"] = vec[2].item()

def setVec6(struct, vec):
    for r, cell in enumerate(["x", "y", "z"]):
        for col, column in enumerate(["c0", "c1"]):
            struct[column][cell] = vec[r, col].item()

def insert_pos(template,
               positions=None, rotations=None, velocity=None,
               tPos=None, tRot=None, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            t = 0
            for j in range(shape[2]):
                jo = template["frames"][f]["joints"][j]
                if positions is not None:
                    setVec3(jo["position"], positions[c,f,j])
                if rotations is not None:
                    setVec3(jo["velocity"], velocity[c,f,j])
                if velocity is not None:
                    setVec6(jo["rotMat"], rotations[c,f,j])


                if jo["key"]:
                    if tPos is not None:
                        setVec3(jo["cost"]["TargetPosition"], tPos[c,f,t])
                    if tRot is not None:
                        setVec6(jo["cost"]["TargetRotation"], tRot[c,f,t])
                    t+=1
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

def generate_animation(model, test_set, feature_dims, template_path, output_path, use_vae=False, n=5):
    idx = np.arange(n)
    with torch.no_grad():
        model.eval()
        model.cpu()
        model.pose_autoencoder.encoder.eval()
        model.pose_autoencoder.decoder.eval()
        model.generationModel.eval()

        x = torch.stack([test_set[i][0] for i in idx])
        y = torch.stack([test_set[i][1] for i in idx])
        shape = x.shape
        x = x.view(-1, shape[-1])

        x.cpu()
        if use_vae:
            out, z, mu, logvar = model(x)
        else:
            out = model(x)
        x_c = torch.cat(out,dim=1).detach()
        generated = x_c
        generated = generated.view(shape)

    phase= feature_dims["phase_vec_l2"]
    toPosDim = phase+feature_dims["pos"]
    toRotDim = toPosDim + feature_dims["rotMat2"]
    toVelDim = toRotDim + feature_dims["velocity"]

    gPos = generated[:, :, phase:toPosDim]
    gRot = generated[:, :, toPosDim:toRotDim]
    gVel = generated[:, :, toRotDim:toVelDim]

    oPos = y[:, :, phase:toPosDim]
    oRot = y[:, :, toPosDim:toRotDim]
    oVel = y[:, :, toRotDim:toVelDim]

    tPos = y[:, :, -target_dim:-target_dim+3*4]
    tRot = y[:, :, -target_dim+3*4:]

    print(gPos.shape, gRot.shape, gVel.shape)
    print(oPos.shape, oRot.shape, oVel.shape)

    clip_length = gPos.shape[1]
    gPos_r = gPos.reshape((n, clip_length, -1, 3))
    gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
    gVel_r = gVel.reshape((n, clip_length, -1, 3))

    oPos_r = oPos.reshape((n, clip_length, -1, 3))
    oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))
    oVel_r = oVel.reshape((n, clip_length, -1, 3))

    tPos_r = tPos.reshape((n, clip_length, -1, 3))
    tRot_r = tRot.reshape((n, clip_length, -1, 3, 3))

    template = js.load(open(template_path))

    insert_pos(template, oPos_r, oRot_r, oVel_r, tPos_r, tRot_r,output_path+"_O")
    insert_pos(template, gPos_r, gRot_r, gVel_r, tPos_r, tRot_r,output_path+"_G")

def generate_animation_ae(model, test_set, feature_dims, template_path, output_path, n=5):
    idx = np.arange(n)
    with torch.no_grad():
        model.eval()
        model.cpu()

        x = torch.stack([test_set[i][0] for i in idx])
        y = torch.stack([test_set[i][1] for i in idx])
        shape = x.shape
        x = x.view(-1, shape[-1])

        x.cpu()
        out = model(x)

        generated = out
        generated = generated.view(shape)

    toPosDim = feature_dims["pos"]
    toRotDim = toPosDim + feature_dims["rotMat2"]
    toVelDim = toRotDim + feature_dims["velocity"]

    gPos = generated[:, :, :toPosDim]
    gRot = generated[:, :, toPosDim:toRotDim]
    gVel = generated[:, :, toRotDim:toVelDim]

    oPos = y[:, :, :toPosDim]
    oRot = y[:, :, toPosDim:toRotDim]
    oVel = y[:, :, toRotDim:toVelDim]

    print(gPos.shape, gRot.shape, gVel.shape)
    print(oPos.shape, oRot.shape, oVel.shape)

    clip_length = gPos.shape[1]
    gPos_r = gPos.reshape((n, clip_length, -1, 3))
    gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
    gVel_r = gVel.reshape((n, clip_length, -1, 3))

    oPos_r = oPos.reshape((n, clip_length, -1, 3))
    oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))
    oVel_r = oVel.reshape((n, clip_length, -1, 3))

    template = js.load(open(template_path))

    insert_pos(template, oPos_r, oRot_r, oVel_r, None, None,output_path+"_O")
    insert_pos(template, gPos_r, gRot_r, gVel_r, None, None, output_path+"_G")